In [ ]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from tensorflow.keras.preprocessing import image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# importing libraries for Deep Learning
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive

# Mount Google Drive để truy cập dữ liệu
drive.mount('/content/drive')

# Đường dẫn tuyệt đối đến thư mục chứa dữ liệu trên Google Drive
base_dir = '/content/drive/My Drive/dataset1'

train_dir = base_dir + '/training_set'
test_dir = base_dir + '/test_set'

train_dir_cats = train_dir + '/cats'
train_dir_dogs = train_dir + '/dogs'
test_dir_cats = test_dir + '/cats'
test_dir_dogs = test_dir + '/dogs'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# Kiểm tra số lượng hình ảnh trong từng thư mục
if os.path.exists(train_dir_cats):
    num_train_cats = len(os.listdir(train_dir_cats))
    print('number of cats training images - ', num_train_cats)
else:
    print('Directory not found:', train_dir_cats)

if os.path.exists(train_dir_dogs):
    num_train_dogs = len(os.listdir(train_dir_dogs))
    print('number of dogs training images - ', num_train_dogs)
else:
    print('Directory not found:', train_dir_dogs)

if os.path.exists(test_dir_cats):
    num_test_cats = len(os.listdir(test_dir_cats))
    print('number of cats testing images - ', num_test_cats)
else:
    print('Directory not found:', test_dir_cats)

if os.path.exists(test_dir_dogs):
    num_test_dogs = len(os.listdir(test_dir_dogs))
    print('number of dogs testing images - ', num_test_dogs)
else:
    print('Directory not found:', test_dir_dogs)

number of cats training images -  2000
number of dogs training images -  2001
number of cats testing images -  1000
number of dogs testing images -  1005


In [ ]:
data_generator = ImageDataGenerator(rescale = 1.0/255.0, zoom_range = 0.2)
#rescale: Scale lại giá trị của các pixel trong hình ảnh bằng cách chia cho 255.0. Điều này giúp đưa giá trị pixel về khoảng từ 0 đến 1, giúp việc huấn luyện hiệu quả hơn.
#zoom_range: Phạm vi cho phép zoom (phóng to hoặc thu nhỏ) hình ảnh. Trong trường hợp này, giá trị 0.2 cho biết hình ảnh có thể được zoom trong khoảng từ 0.8 đến 1.2 lần kích thước gốc.

In [ ]:
batch_size = 32
training_data = data_generator.flow_from_directory(directory = train_dir,
                                                   target_size = (64, 64),
                                                   batch_size = batch_size,
                                                   class_mode = 'binary')
testing_data = data_generator.flow_from_directory(directory = test_dir,
                                                  target_size = (64, 64),
                                                  batch_size = batch_size,
                                                  class_mode = 'binary')

Found 4001 images belonging to 2 classes.
Found 2005 images belonging to 2 classes.


In [ ]:
# preparing the layers in the Convolutional Deep Neural Network
model = Sequential() #tạo ra mô hình tuần tự
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = training_data.image_shape))
#Layer này sẽ nhận đầu vào là hình ảnh và áp dụng 32 bộ lọc với kích thước kernel là (3, 3). Hàm kích hoạt được sử dụng là ReLU (Rectified Linear Unit).

model.add(MaxPooling2D(pool_size = (2, 2))) # giảm kích thước của đầu ra
model.add(Dropout(rate = 0.3)) # giảm hiện tượng quá khớp (overfitting) bằng cách ngẫu nhiên bỏ qua một phần các đơn vị đầu ra
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.2)) # như trên
model.add(Conv2D(filters = 126, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.15)) # y vậy
model.add(Flatten()) #đổi đầu vào từ tensor 2D thành vector 1D để chuẩn bị cho các layer fully connected (hoàn toàn kết nối)
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dropout(rate = 0.15)) #giảm quá khớp
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(rate = 0.1)) #y vậy
model.add(Dense(units = len(set(training_data.classes)), activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 31, 31, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 14, 14, 64)        0

In [ ]:
# Đoạn code  sử dụng phương thức fit() để huấn luyện mô hình neural network.
fitted_model = model.fit(training_data,
                        steps_per_epoch = 100,
                        epochs = 10,
                        validation_data = testing_data,
                        validation_steps = 15)


Epoch 1/10
100/100 [==============================] - 265s 3s/step - loss: 0.6923 - accuracy: 0.5134 - val_loss: 0.6939 - val_accuracy: 0.4792
Epoch 2/10
100/100 [==============================] - 189s 2s/step - loss: 0.6939 - accuracy: 0.5039 - val_loss: 0.6916 - val_accuracy: 0.5167
Epoch 3/10
100/100 [==============================] - 118s 1s/step - loss: 0.6902 - accuracy: 0.5412 - val_loss: 0.6783 - val_accuracy: 0.5646
Epoch 4/10
100/100 [==============================] - 115s 1s/step - loss: 0.6706 - accuracy: 0.5876 - val_loss: 0.6859 - val_accuracy: 0.5396
Epoch 5/10
100/100 [==============================] - 77s 771ms/step - loss: 0.6591 - accuracy: 0.6157 - val_loss: 0.6340 - val_accuracy: 0.6687
Epoch 6/10
100/100 [==============================] - 76s 757ms/step - loss: 0.6523 - accuracy: 0.6153 - val_loss: 0.6711 - val_accuracy: 0.5771
Epoch 7/10
100/100 [==============================] - 75s 757ms/step - loss: 0.6493 - accuracy: 0.6212 - val_loss: 0.6419 - val_accuracy: 

In [ ]:

# Giả sử bạn có độ chính xác và độ chính xác xác thực được lưu trữ trong danh sách hoặc mảng
accuracy = [0.85, 0.9, 0.92, 0.95, 0.97]
val_accuracy = [0.82, 0.88, 0.89, 0.92, 0.94]

epochs = range(1, len(accuracy) + 1)

# Độ chính xác của dự đoán và độ chính xác xác nhận
plt.plot(epochs, accuracy, 'bo', label='Độ chính xác huấn luyện')
plt.plot(epochs, val_accuracy, 'r', label='Độ chính xác thực tế')
plt.title('Độ chính xác huấn luyện và độ chính xác thực tế')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# testing the model
def testing_image(image_directory):
    test_image = image.load_img(image_directory, target_size = (64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = model.predict(x = test_image)
    print(result)
    if result[0][0]  == 1:
        prediction = 'Dog'
    else:
        prediction = 'Cat'
    return prediction

**Kết quả đạt được**
Đường dẫn test ảnh có thể thay đổi 2 dạng là:
+ *'/dogs/dog.4001.jpg'*  (Số đây có thể thay đổi từ 4001 đến 5000)
+ *'/cats/cat.4001.jpg'*  (Số đây có thể thay đổi từ 4001 đến 5000)

**Kết qua ta sẽ có dạng :**
[[Tỉ lệ giống 'chó' và Tỉ lệ giống 'mèo']]
Kết luận loài nào

In [ ]:
model.save('/content/gdrive/MyDrive/my_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
print(testing_image(test_dir + '/dogs/dog.4500.jpg'))

1/1 [==============================] - 0s 34ms/step
[[1. 0.]]
Dog
